In [135]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv(r'C:\Users\aguil\Downloads\P_Data_Extract_From_World_Development_Indicators.zip')

# Mostrar las primeras filas del DataFrame
print(df.head())

                         Series Name  Series Code    Country Name  \
0  Consumer price index (2010 = 100)  FP.CPI.TOTL     Afghanistan   
1  Consumer price index (2010 = 100)  FP.CPI.TOTL         Albania   
2  Consumer price index (2010 = 100)  FP.CPI.TOTL         Algeria   
3  Consumer price index (2010 = 100)  FP.CPI.TOTL  American Samoa   
4  Consumer price index (2010 = 100)  FP.CPI.TOTL         Andorra   

  Country Code   1990 [YR1990]     2000 [YR2000]     2015 [YR2015]  \
0          AFG              ..                ..  132.883209191418   
1          ALB              ..  74.1374705168667  113.150704053254   
2          DZA  15.51936355384  70.4381279952933  126.736646357875   
3          ASM              ..                ..                ..   
4          AND              ..                ..                ..   

      2016 [YR2016]     2017 [YR2017]     2018 [YR2018]     2019 [YR2019]  \
0  138.708665508891  145.610741438593  146.522481857209  149.895975208032   
1  112.735

In [136]:
import pandas as pd
import numpy as np

# Cargar y limpiar datos originales (reemplaza ".." con NaN)
df = pd.read_csv(
    r'C:\Users\aguil\Downloads\P_Data_Extract_From_World_Development_Indicators.zip',
    compression='zip',
    na_values=".."  # Convierte ".." a NaN automáticamente
)

# Identificar columnas de años
year_columns = [col for col in df.columns if '[YR' in col]

# Reorganizar datos 
df_tidy = pd.melt(
    df,
    id_vars=['Series Name', 'Series Code', 'Country Name', 'Country Code'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

# Extraer el año numérico (ej: "2018 [YR2018]" → 2018)
df_tidy['Year'] = df_tidy['Year'].str.extract(r'(\d{4})').astype(int)

# Filtrar solo 2018 y eliminar filas con NaN 
df_2018_clean = df_tidy[
    (df_tidy['Year'] == 2018) & 
    (df_tidy['Value'].notna())  # Solo filas con valores numéricos
].reset_index(drop=True)

# Mostrar resultados
print(df_2018_clean)

                           Series Name  Series Code         Country Name  \
0    Consumer price index (2010 = 100)  FP.CPI.TOTL          Afghanistan   
1    Consumer price index (2010 = 100)  FP.CPI.TOTL              Albania   
2    Consumer price index (2010 = 100)  FP.CPI.TOTL              Algeria   
3    Consumer price index (2010 = 100)  FP.CPI.TOTL               Angola   
4    Consumer price index (2010 = 100)  FP.CPI.TOTL  Antigua and Barbuda   
..                                 ...          ...                  ...   
176  Consumer price index (2010 = 100)  FP.CPI.TOTL              Vanuatu   
177  Consumer price index (2010 = 100)  FP.CPI.TOTL             Viet Nam   
178  Consumer price index (2010 = 100)  FP.CPI.TOTL   West Bank and Gaza   
179  Consumer price index (2010 = 100)  FP.CPI.TOTL               Zambia   
180  Consumer price index (2010 = 100)  FP.CPI.TOTL             Zimbabwe   

    Country Code  Year       Value  
0            AFG  2018  146.522482  
1            

In [137]:
import pandas as pd

# Seleccionar solo las columnas requeridas: Country Name, 2018 y Series Name (CPI)
df_2018_cpi = df[['Country Name', '2018 [YR2018]']]

# Renombrar columnas 
df_2018_cpi = df_2018_cpi.rename(columns={
    'Country Name': 'Country',
    '2018 [YR2018]': 'CPI_2018',
    
})

# Eliminar filas con valores faltantes (NaN) en CPI_2018
df_2018_cpi_clean = df_2018_cpi.dropna(subset=['CPI_2018'])

# Mostrar el resultado
print(df_2018_cpi_clean)

                 Country    CPI_2018
0            Afghanistan  146.522482
1                Albania  117.391546
2                Algeria  148.463995
5                 Angola  324.218068
6    Antigua and Barbuda  113.807620
..                   ...         ...
209              Vanuatu  113.980923
211             Viet Nam  159.069641
213   West Bank and Gaza  110.765815
215               Zambia  194.510438
216             Zimbabwe  116.712211

[181 rows x 2 columns]


CPI de cada pais en 2018


In [138]:
import pandas as pd
from zipfile import ZipFile
import io

# Ruta al archivo ZIP
zip_path = r'C:\Users\aguil\Downloads\API_SL.UEM.TOTL.NE.ZS_DS2_es_csv_v2_30517 (1).zip'

with ZipFile(zip_path) as z:
    # Listar archivos disponibles 
    print("Archivos en el ZIP:", z.namelist())
    
    # Extraer y leer el archivo principal
    with z.open('API_SL.UEM.TOTL.NE.ZS_DS2_es_csv_v2_30517.csv') as f:
        # Leer como DataFrame 
        df = pd.read_csv(io.BytesIO(f.read()), encoding='utf-8', header=4)
        
print(df)

Archivos en el ZIP: ['Metadata_Indicator_API_SL.UEM.TOTL.NE.ZS_DS2_es_csv_v2_30517.csv', 'API_SL.UEM.TOTL.NE.ZS_DS2_es_csv_v2_30517.csv', 'Metadata_Country_API_SL.UEM.TOTL.NE.ZS_DS2_es_csv_v2_30517.csv']
          Unnamed: 0  AFE  \
0         Afganistán  AFG   
1                NaN  AFW   
2             Angola  AGO   
3            Albania  ALB   
4            Andorra  AND   
..               ...  ...   
259           Kosovo  XKX   
260  Yemen, Rep. del  YEM   
261        Sudáfrica  ZAF   
262           Zambia  ZMB   
263         Zimbabwe  ZWE   

    Desempleo, total (% de participación total en la fuerza laboral) (estimación nacional)  \
0    Desempleo, total (% de participación total en ...                                       
1    Desempleo, total (% de participación total en ...                                       
2    Desempleo, total (% de participación total en ...                                       
3    Desempleo, total (% de participación total en ...                 

In [139]:
print("Columnas disponibles:", df.columns.tolist())

Columnas disponibles: ['Unnamed: 0', 'AFE', 'Desempleo, total (% de participación total en la fuerza laboral) (estimación nacional)', 'SL.UEM.TOTL.NE.ZS', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 6

In [140]:
# Seleccionar columnas específicas para 2018
df_2018 = df[['Unnamed: 0','Unnamed: 62' ]].copy()

# Renombrar columnas
df_2018.columns = ['País', 'Desempleo total']

# Limpiar datos 
df_2018 = df_2018.dropna()

# Mostrar resultados
print("Desempleo total en 2018 (% de la fuerza laboral):")
df_2018



Desempleo total en 2018 (% de la fuerza laboral):


,País,Desempleo total
3,Albania,12.304000
6,Emiratos Árabes Unidos,2.236000
7,Argentina,9.221000
8,Armenia,10.589000
10,Antigua y Barbuda,8.780000
...,...,...
255,Viet Nam,1.161000
257,Mundo,5.978452
259,Kosovo,27.761000
261,Sudáfrica,26.906000


Desempleo 2018

In [141]:
import pandas as pd
from zipfile import ZipFile
import io

# Ruta al archivo ZIP
zip_path = r'C:\Users\aguil\Downloads\API_FR.INR.RINR_DS2_en_csv_v2_19336.zip'

with ZipFile(zip_path) as z:
    # Listar archivos disponibles 
    print("Archivos en el ZIP:", z.namelist())
    
    # Extraer y leer el archivo principal
    with z.open('API_FR.INR.RINR_DS2_en_csv_v2_19336.csv') as f:
        # Leer como DataFrame 
        df = pd.read_csv(io.BytesIO(f.read()), encoding='utf-8', header=4)
        
print(df)

Archivos en el ZIP: ['Metadata_Indicator_API_FR.INR.RINR_DS2_en_csv_v2_19336.csv', 'API_FR.INR.RINR_DS2_en_csv_v2_19336.csv', 'Metadata_Country_API_FR.INR.RINR_DS2_en_csv_v2_19336.csv']
    Africa Eastern and Southern  AFE  Real interest rate (%)  FR.INR.RINR  \
0                   Afghanistan  AFG  Real interest rate (%)  FR.INR.RINR   
1    Africa Western and Central  AFW  Real interest rate (%)  FR.INR.RINR   
2                        Angola  AGO  Real interest rate (%)  FR.INR.RINR   
3                       Albania  ALB  Real interest rate (%)  FR.INR.RINR   
4                       Andorra  AND  Real interest rate (%)  FR.INR.RINR   
..                          ...  ...                     ...          ...   
259                      Kosovo  XKX  Real interest rate (%)  FR.INR.RINR   
260                 Yemen, Rep.  YEM  Real interest rate (%)  FR.INR.RINR   
261                South Africa  ZAF  Real interest rate (%)  FR.INR.RINR   
262                      Zambia  ZMB  Real i

In [142]:
print("Columnas disponibles:", df.columns.tolist())

Columnas disponibles: ['Africa Eastern and Southern', 'AFE', 'Real interest rate (%)', 'FR.INR.RINR', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unn

In [143]:
# Seleccionar columnas específicas para 2018
df_2018 = df[['Africa Eastern and Southern','Unnamed: 62' ]].copy()

# Renombrar columnas
df_2018.columns = ['País', 'Tasas de interes']

# Limpiar datos (eliminar filas sin datos)
df_2018 = df_2018.dropna()

# Mostrar resultados
print("Desempleo total en 2018 (% de la fuerza laboral):")
df_2018




Desempleo total en 2018 (% de la fuerza laboral):


,País,Tasas de interes
2,Angola,-5.844003
3,Albania,2.880525
7,Argentina,4.564409
8,Armenia,9.733977
10,Antigua and Barbuda,7.408063
...,...,...
258,Samoa,7.767265
259,Kosovo,5.079009
261,South Africa,5.856184
262,Zambia,2.215865


Tasas de interés

In [144]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv(r'C:\Users\aguil\Downloads\P_Data_Extract_From_World_Development_Indicators (1).zip')

# Mostrar las primeras filas del DataFrame
print(df.head())

                                         Series Name     Series Code  \
0  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
1  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
2  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
3  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
4  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   

     Country Name Country Code     1990 [YR1990]     2000 [YR2000]  \
0     Afghanistan          AFG                ..                ..   
1         Albania          ALB        1271486850  2879464404.75529   
2         Algeria          DZA  35266505182.0216  22776943098.6469   
3  American Samoa          ASM                ..                ..   
4         Andorra          AND                ..                ..   

      2015 [YR2015]     2016 [YR2016]     2017 [YR2017]     2018 [YR2018]  \
0                ..                ..                ..              

In [145]:
import pandas as pd
import numpy as np

# Identificar columnas de años
year_columns = [col for col in df.columns if '[YR' in col]

# Reorganizar datos 
df_tidy = pd.melt(
    df,
    id_vars=['Series Name', 'Series Code', 'Country Name', 'Country Code'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

# Extraer el año numérico 
df_tidy['Year'] = df_tidy['Year'].str.extract(r'(\d{4})').astype(int)

# Filtrar solo 2018 y eliminar filas con NaN
df_2018_clean = df_tidy[
    (df_tidy['Year'] == 2018) & 
    (df_tidy['Value'].notna())  # Solo filas con valores numéricos
].reset_index(drop=True)

# Mostrar resultados
print(df_2018_clean)

                                           Series Name     Series Code  \
0    Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
1    Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
2    Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
3    Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
4    Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
..                                                 ...             ...   
261  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
262  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
263  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
264  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   
265  Households and NPISHs Final consumption expend...  NE.CON.PRVT.CD   

                                   Country Name Country Code  Year  \
0                                   Afgha

In [146]:
# Seleccionar solo las columnas requeridas: Country Name, 2018 y Series Name (CPI)
df_2018_cpi = df[['Country Name', '2018 [YR2018]']]

# Renombrar columnas para mayor claridad
df_2018_cpi = df_2018_cpi.rename(columns={
    'Country Name': 'Country',
    '2018 [YR2018]': 'Consumo Total Hogares 2018',
    
})

# Eliminar filas con valores faltantes (NaN) en CPI_2018
df_2018_cpi_clean = df_2018_cpi.dropna(subset=['Consumo Total Hogares 2018'])

# Mostrar el resultado
print(df_2018_cpi_clean)

                                        Country Consumo Total Hogares 2018
0                                   Afghanistan                         ..
1                                       Albania           11453757185.9689
2                                       Algeria           81186389409.1377
3                                American Samoa                  487000000
4                                       Andorra                         ..
..                                          ...                        ...
261                          Sub-Saharan Africa           1307276997301.39
262  Sub-Saharan Africa (excluding high income)           1305998845590.73
263   Sub-Saharan Africa (IDA & IBRD countries)           1307276997301.39
264                         Upper middle income           11096981141650.3
265                                       World           48743986662133.8

[266 rows x 2 columns]


Consumo total hogares

In [147]:
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv(r'C:\Users\aguil\Downloads\P_Data_Extract_From_World_Development_Indicators (2).zip')

# Mostrar las primeras filas del DataFrame
print(df.head())

         Series Name     Series Code    Country Name Country Code  \
0  GNI (current US$)  NY.GNP.MKTP.CD     Afghanistan          AFG   
1  GNI (current US$)  NY.GNP.MKTP.CD         Albania          ALB   
2  GNI (current US$)  NY.GNP.MKTP.CD         Algeria          DZA   
3  GNI (current US$)  NY.GNP.MKTP.CD  American Samoa          ASM   
4  GNI (current US$)  NY.GNP.MKTP.CD         Andorra          AND   

      1990 [YR1990]     2000 [YR2000]     2015 [YR2015]     2016 [YR2016]  \
0                ..  3411145674.07204  19249499438.7455  18350475875.2999   
1        2026653750  3585007221.82101  11521811393.6203   12053832974.203   
2  59780577656.1963  52080377793.2312  183649437186.975  179594281655.137   
3                ..                ..                ..                ..   
4                ..                ..                ..                ..   

      2017 [YR2017]     2018 [YR2018]     2019 [YR2019]     2020 [YR2020]  \
0  18961769787.4548  18294513957.5523  190806

In [148]:
import pandas as pd
import numpy as np

# Identificar columnas de años
year_columns = [col for col in df.columns if '[YR' in col]

# Reorganizar datos 
df_tidy = pd.melt(
    df,
    id_vars=['Series Name', 'Series Code', 'Country Name', 'Country Code'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Value'
)

# Extraer el año numérico 
df_tidy['Year'] = df_tidy['Year'].str.extract(r'(\d{4})').astype(int)

# Filtrar solo 2018 y eliminar filas con NaN 
df_2018_clean = df_tidy[
    (df_tidy['Year'] == 2018) & 
    (df_tidy['Value'].notna())  # Solo filas con valores numéricos
].reset_index(drop=True)

# Mostrar resultados
print(df_2018_clean)

           Series Name     Series Code  \
0    GNI (current US$)  NY.GNP.MKTP.CD   
1    GNI (current US$)  NY.GNP.MKTP.CD   
2    GNI (current US$)  NY.GNP.MKTP.CD   
3    GNI (current US$)  NY.GNP.MKTP.CD   
4    GNI (current US$)  NY.GNP.MKTP.CD   
..                 ...             ...   
261  GNI (current US$)  NY.GNP.MKTP.CD   
262  GNI (current US$)  NY.GNP.MKTP.CD   
263  GNI (current US$)  NY.GNP.MKTP.CD   
264  GNI (current US$)  NY.GNP.MKTP.CD   
265  GNI (current US$)  NY.GNP.MKTP.CD   

                                   Country Name Country Code  Year  \
0                                   Afghanistan          AFG  2018   
1                                       Albania          ALB  2018   
2                                       Algeria          DZA  2018   
3                                American Samoa          ASM  2018   
4                                       Andorra          AND  2018   
..                                          ...          ...   ...   
261  

In [149]:
# Seleccionar solo las columnas requeridas: Country Name, 2018 y Series Name (CPI)
df_2018_cpi = df[['Country Name', '2018 [YR2018]']]

# Renombrar columnas 
df_2018_cpi = df_2018_cpi.rename(columns={
    'Country Name': 'Country',
    '2018 [YR2018]': 'Ingreso nacional bruto 2018',
    
})

# Eliminar filas con valores faltantes (NaN) en CPI_2018
df_2018_cpi_clean = df_2018_cpi.dropna(subset=['Ingreso nacional bruto 2018'])

# Mostrar el resultado
print(df_2018_cpi_clean)

                                        Country Ingreso nacional bruto 2018
0                                   Afghanistan            18294513957.5523
1                                       Albania            15361070727.3542
2                                       Algeria            190390454064.656
3                                American Samoa                          ..
4                                       Andorra                          ..
..                                          ...                         ...
261                          Sub-Saharan Africa             1730435953736.8
262  Sub-Saharan Africa (excluding high income)            1728736809088.67
263   Sub-Saharan Africa (IDA & IBRD countries)             1730435953736.8
264                         Upper middle income              23044511357732
265                                       World            86654073045328.3

[266 rows x 2 columns]


Ingreso Nacional Bruto